# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

# Read Excelfiles
## Phishing report

### In order for this to work, save all reports in the Data folder

In [2]:
xl_file = pd.ExcelFile('Data/DataPhishingReport.xlsx', engine='openpyxl')

In [3]:
# Read every sheet within the spreadsheet
dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}

In [4]:
# Exclude the sheet that is named 'Blank' as it does not seem to be useful for the data analysis
for df_name, df_content in dfs.items():
    if df_name != 'Blank':
        df_content['Type'] = df_name

In [5]:
real_phishing = dfs['RealPhishing']
attachments = dfs['Attachments']
internal = dfs['Internal Phishing']
spam = dfs['Spam']

In [6]:
# Standardize fields across all sheets
real_phishing['Sender'] = real_phishing['UserID']
real_phishing = real_phishing.rename(columns={"Date": "Received"})
real_phishing = real_phishing.drop(['Unnamed: 1', 'Unnamed: 2', 'UserID', 'More than 1 recipient', 'TicketNr'], axis=1)

In [7]:
sender = real_phishing.pop('Sender')
subject = real_phishing.pop('Subject')
real_phishing.insert(0, 'Subject', subject)
real_phishing.insert(2, 'Sender', sender)
real_phishing.head()

,Subject,Received,Sender,Type
0,Phishing Email Report: Delivery restriction no...,2020-09-14,9903746.0,RealPhishing
1,Phishing Email Report: ProposaIs from TORROJA,2020-09-14,9918069.0,RealPhishing
2,"Phishing Email Report: [MARKETING] Tyler, Your...",2020-09-14,10823974.0,RealPhishing
3,Phishing Email Report: Your mindy.flaherty@hil...,2020-09-14,9938174.0,RealPhishing
4,Phishing Email Report: Your mindy.flaherty@hil...,2020-09-15,9938174.0,RealPhishing


In [8]:
# Concatenate all sheets into one phishing report
phishing_report = pd.concat([real_phishing, attachments, internal, spam])

In [9]:
grouped = phishing_report.groupby(['Sender', 'Type']).count().copy().reset_index()

In [10]:
# Check for missing User IDs. This is the main information for us, so drop them (if there is not too many)
phishing_report.loc[phishing_report['Sender'].isnull()]

,Subject,Received,Sender,Type
22,"Phishing Email Report: 21 September, 2020, 𝚎Vo...",2020-09-22 00:00:00,NaN,RealPhishing
26,"Phishing Email Report: 22 September, 2020, 𝚎Vo...",2020-09-22 00:00:00,NaN,RealPhishing
73,Phishing Email Report: Issues with the phone s...,2020-09-22 18:53:15,NaN,Spam


In [11]:
# Enter indexes with missing values inside drop([])
phishing_report = phishing_report.drop([22, 26, 73]).copy()

In [12]:
# Make sure User IDs are formatted as integers
phishing_report['Sender'] = phishing_report['Sender'].astype(int)

In [13]:
grouped.columns

Index(['Sender', 'Type', 'Subject', 'Received'], dtype='object')

In [14]:
# Data exploration
grouped.loc[(grouped['Type'] == 'Spam') & (grouped['Received'] > 10)]

,Sender,Type,Subject,Received
30,9904608.0,Spam,12,12
31,9904646.0,Spam,12,12
40,9917492.0,Spam,13,13
93,9922294.0,Spam,21,21
102,9922778.0,Spam,18,18
159,9925734.0,Spam,11,11
176,9926897.0,Spam,32,32
203,9928743.0,Spam,53,53
279,9935944.0,Spam,14,14
289,9936746.0,Spam,37,37


## Cybersecurity Awareness Training

In [15]:
training = pd.read_excel('Data/IT Security Awareness Global report.xlsx', engine='openpyxl')

## There are a lot of columns in the report, not all are useful

In [16]:
training = training.drop(['Course specified approver', 'Username', 'FunctionalAreaCode', 'CostCenterName', 
                          'Time spent on test', 'Suspend data', 'Creator approver', 'Manager approver', 'Specified Approver', 'Approval date', 
                          'Community name', 'EmployeeGuid', 'SAPCentralPersonNumber', 'Email', 'Content ID', 'TradeAssignment(Attribute8)', 
                          'SalesOrganizationName', 'SalesOrganizationCode', 'Last login', 'Last activity'], axis=1)

In [17]:
# Calculate the difference between now and the date when the course was completed for "time passed since" score
import datetime
now = datetime.datetime.now().replace(microsecond=0)
diff = (now - training['Date course completed']).dt.days

In [18]:
training['Time passed since'] = diff

In [19]:
training['SCORM course status'].unique()

array(['passed', 'completed', 'not started', 'incomplete', 'failed'],
      dtype=object)

In [20]:
# Drop users with missing valuable data
training = training.drop([18607, 18924, 22366, 22356, 24359, 24440, 25524, 26464, 28813, 29075, 29171, 29591, 30199, 31151, 32124])

In [21]:
# Reformat data
grouped_training = training.groupby(['Department', 'SCORM course status', 'Test score', 'User ID']).count().copy()

In [22]:
top_departments_low_score = training.loc[(training['Test score'] < 50) & (training['Test score'] > 0)].groupby('Department')['User ID'].count().reset_index().sort_values('User ID', ascending=False).copy()

In [23]:
# Some data exploration
top_departments_low_score

,Department,User ID
0,AREA 143,2
35,K Executive Board,2
10,Dayton Inventory,2
39,ME Ost,2
41,MO FR Local Support,2
...,...,...
25,Global Logistics Services,1
24,General Ledger,1
23,Finance Operations,1
22,FRANCE 1,1


In [24]:
training.loc[training['SCORM course status'].isin(['not started', 'failed', 'incomplete'])].groupby('Department')['User ID'].count().reset_index().sort_values('User ID', ascending=False).copy()

,Department,User ID
1481,Sales,111
1088,ODO Operations Processes,108
287,DSG Software Process Governance,94
1256,P88 Tool Production 1,62
1291,Production,53
...,...,...
961,META Talent Aquisition and Management,1
954,ME North West,1
950,MCS,1
949,MCC Digital,1


In [25]:
# Completed should only mean successful completion. If they failed, then change the status to 'failed'
training.loc[(training['SCORM course status'] == 'completed') & (training['Test score'] < 80), 'SCORM course status'] = 'failed'

## Phishing simulation test

In [26]:
simulation = pd.read_excel('FirstPhishingSimulation_Results.xlsx', engine='openpyxl')

In [27]:
# Drop unnecessary columns
simulation = simulation.drop(['Received Emails', 'Send Email', 'Recipients', 'Time Zone', 'Clicked Link', 'Completed Form', 'Visited Feedback Page', 'Department2', 'Total Education Time', 'Department3'], axis=1)

In [28]:
simulation.loc[(simulation['Opened Attachment']==1) & (simulation['Visited Feedback Page True False'] == 0)]

,Scenario,UserID,Language,Opened Emails,Reported Emails,Viewed Images,Clicked Link True/False,Opened Attachment,Completed Form True False,Visited Feedback Page True False,IsoCode,Department1,Company,Position,Department


In [29]:
# Deducting some missing data
simulation.loc[simulation['Department1'].isnull(), 'Department1'] = simulation['Company']
simulation.loc[simulation['IsoCode'].isnull(), 'IsoCode'] = simulation['Company']

In [30]:
simulation.loc[(simulation['IsoCode'].isnull()) & (simulation['Department1'].isnull()) & (simulation['IsoCode'].isnull())].index

Int64Index([  105,  1731,  4424,  4491,  4591,  7914, 10928, 11231, 11496,
            12077, 13469, 16586, 16587, 19067, 21501, 22236, 22770, 22784,
            27809, 27844],
           dtype='int64')

In [31]:
# Drop rows with missing values that cannot be deducted
simulation = simulation.drop([105, 1731, 4424, 4491, 4591, 7914, 10928, 11231, 11496, 12077, 13469, 16586, 16587, 19067, 21501, 22236, 22770, 22784, 27809, 27844])

In [32]:
# 161 people failed the test and reported the simulation email
simulation.loc[(simulation['Completed Form True False'] == 1) & (simulation['Reported Emails'] == 1)]

,Scenario,UserID,Language,Opened Emails,Reported Emails,Viewed Images,Clicked Link True/False,Opened Attachment,Completed Form True False,Visited Feedback Page True False,IsoCode,Department1,Company,Position,Department
13,Nov 24 Phishing Sim,10690548,English,1,1,1,1,0,1,1,IN,sales,Hilti India Pvt. Ltd.,Sr. Technical Account Manager,Area Sales - Gen East Central
18,Nov 24 Phishing Sim,9940169,English,1,1,1,1,0,1,1,IN,sales,Hilti India Pvt. Ltd.,Key Account Manager,Area Sales - EI South 2
501,Nov 24 Phishing Sim,9935354,English,1,1,0,1,0,1,1,MY,info.technology / info.services,Hilti Asia IT Services Sdn Bhd,Head of Core Infrastructure Services,FII IT Infrastructure
781,Nov 24 Phishing Sim,10354058,English,1,1,1,1,0,1,1,LI,product portfolio management,hilti ag,Supply Manager,XV Supply Management DX
889,Nov 24 Phishing Sim,9935126,English,1,1,0,1,0,1,1,IN,general management,hilti manufacturing india,Chief Technical Officer,ZEC Diamond Insert Tools
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30868,Nov 24 Phishing Sim,11263702,English,1,1,0,1,0,1,1,IN,finance,Hilti India Pvt. Ltd.,Sr. Business Analyst,FCC Business Reporting
30976,Nov 24 Phishing Sim,10647914,English,1,1,1,1,0,1,1,CN,manufacturing,Hilti (Shanghai) Ltd.,Management Associate-Logistic,P88 Logistics
31087,Nov 24 Phishing Sim,9927510,English,1,1,1,1,0,1,1,SA,sales,hilti saudi arabia,Head of Key Project Managers,KA-KP Sales
31449,Nov 24 Phishing Sim,10893500,English,1,1,1,1,0,1,1,CN,product portfolio management,Hilti (Shanghai) Ltd.,Supply Manager,HSA Supply-AlliedInserts


In [33]:
simulation.shape

(31666, 15)

In [34]:
# More country deduction
simulation.loc[simulation['IsoCode'] == 'Hilti Jordan', 'IsoCode'] = 'JO'
simulation = simulation.drop([9584, 15063, 17155, 29028])
simulation.loc[simulation['IsoCode'].str.len() > 2, 'IsoCode'] = ''
sim_countries = dict(zip(simulation['UserID'], simulation['IsoCode']))
user_countries = dict(zip(training['User ID'], training['CountryISOCode']))

## Preprocess data for the combined report

In [35]:
# Get ids from every report
report_ids = np.array(phishing_report['Sender'])
training_ids = np.array(training['User ID'])
simulation_ids = np.array(simulation['UserID'])

In [36]:
# Only save unique ids
unique_ids = np.unique(np.concatenate((report_ids, training_ids, simulation_ids), 0))

### Get relevant data from each report by User ID

In [37]:
users_failed_sim = dict(zip(simulation['UserID'], simulation['Completed Form True False']))

In [38]:
passed_training = dict(zip(training['User ID'], training['SCORM course status']))

In [39]:
time_since_training = dict(zip(training['User ID'], training['Time passed since']))

In [40]:
training_score = dict(zip(training['User ID'], training['Test score']))

In [41]:
report_anything = phishing_report['Sender']

In [42]:
users_sim_1 = dict(zip(simulation['UserID'], simulation['Viewed Images']))
users_sim_2 = dict(zip(simulation['UserID'], simulation['Clicked Link True/False']))
users_sim_3 = dict(zip(simulation['UserID'], simulation['Opened Attachment']))

In [43]:
users_report_sim = dict(zip(simulation['UserID'], simulation['Reported Emails']))

In [44]:
users_watch_sim = dict(zip(simulation['UserID'], simulation['Visited Feedback Page True False']))

In [45]:
'''
Users completed form (0: did not submit data in simulation, 1: submitted data), 
passed training (0: passed/completed, 1: not passed/completed)
reported anything via phishing report button: (0: no, 1: yes)
test score
time since training: 1 = low(less than 6 months), 2 = medium(more than 6 months less than 1.5), 3 = high(more than 1.5 years)
did user view images in simulation (0: no, 1: yes)
did user click link in simulation (0: no, 1: yes)
did user open attachment in simulation (0: no, 1: yes)
did user report: -1 yes, +1 no
did user watch feedback - (0: no, -1: yes)
'''

'\nUsers completed form (0: did not submit data in simulation, 1: submitted data), \npassed training (0: passed/completed, 1: not passed/completed)\nreported anything via phishing report button: (0: no, 1: yes)\ntest score\ntime since training: 1 = low(less than 6 months), 2 = medium(more than 6 months less than 1.5), 3 = high(more than 1.5 years)\ndid user view images in simulation (0: no, 1: yes)\ndid user click link in simulation (0: no, 1: yes)\ndid user open attachment in simulation (0: no, 1: yes)\ndid user report: -1 yes, +1 no\ndid user watch feedback - (0: no, -1: yes)\n'

### Append all data into a dictionary, where key is a user id and values are each value for each feature

In [46]:
temp = {}
for uid in unique_ids:
    try:
        temp[uid] = [users_failed_sim[uid]]
    except KeyError:
        temp[uid] = [0]

In [47]:
for uid in unique_ids:
    try:
        if passed_training[uid] == 'passed' or passed_training[uid] == 'completed':
            temp[uid].append(0)
        else:
            temp[uid].append(1)
    except KeyError:
        temp[uid].append(1)

In [48]:
for uid in unique_ids:
    if uid in phishing_report['Sender']:
        temp[uid].append(1)
    else:
        temp[uid].append(0)

In [49]:
for uid in unique_ids:
    try:
        temp[uid].append(training_score[uid])
    except KeyError:
        temp[uid].append(0)

In [50]:
low = 183
high = 548

In [51]:
for k,v in time_since_training.items():
    if str(v) == 'nan':
        metric = 3
    elif v <= low:
        metric = 1
    elif v > low and v <= high:
        metric = 2
    else:
        metric = 3
    time_since_training[k] = metric  

In [52]:
for uid in unique_ids:
    try:
        temp[uid].append(time_since_training[uid])
    except KeyError:
        temp[uid].append(3)

In [53]:
for uid in unique_ids:
    for d in [users_sim_1, users_sim_2, users_sim_3]:
        try:
            temp[uid].append(d[uid])
        except KeyError:
            temp[uid].append(0)

In [54]:
for uid in unique_ids:
    try:
        if users_report_sim[uid] == 1 and temp[uid][0] == 0 and temp[uid][5] == 0 and temp[uid][6] == 0 and temp[uid][7] == 0:
            temp[uid].append(-1)
        else:
            temp[uid].append(1)
    except KeyError:
        temp[uid].append(1)

In [55]:
for uid in unique_ids:
    try:
        if users_watch_sim[uid] == 1:
            temp[uid].append(-1)
        else:
            temp[uid].append(0)
    except KeyError:
        temp[uid].append(0)

In [56]:
green = 4
yellow = 7
red = 9

In [57]:
# End result
temp

{6677201: [0, 0, 0, 100.0, 3, 0, 0, 0, 1, 0],
 6812989: [0, 0, 0, 100.0, 3, 0, 0, 0, 1, 0],
 6812994: [0, 0, 0, 100.0, 2, 0, 1, 0, 1, 0],
 7469741: [0, 0, 0, 100.0, 3, 0, 0, 0, 1, 0],
 8577487: [1, 0, 0, 100.0, 3, 1, 1, 0, 1, -1],
 8819324: [1, 0, 0, 100.0, 2, 1, 1, 0, 1, -1],
 8924410: [0, 0, 0, 100.0, 1, 1, 1, 0, 1, -1],
 8927352: [1, 0, 0, 100.0, 3, 1, 1, 0, 1, -1],
 8941913: [0, 0, 0, 100.0, 2, 0, 0, 0, 1, 0],
 9374413: [1, 0, 0, 100.0, 2, 0, 1, 0, 1, -1],
 9374414: [0, 0, 0, 100.0, 3, 0, 0, 0, 1, 0],
 9374415: [0, 0, 0, 100.0, 3, 1, 1, 0, 1, 0],
 9374416: [0, 0, 0, 100.0, 3, 1, 0, 0, 1, 0],
 9374561: [0, 1, 0, nan, 3, 0, 0, 0, 1, 0],
 9399474: [0, 0, 0, 100.0, 3, 0, 0, 0, -1, 0],
 9678721: [0, 0, 0, 100.0, 2, 1, 1, 0, 1, 0],
 9684128: [1, 0, 0, 100.0, 2, 1, 1, 0, 1, -1],
 9688411: [0, 0, 0, 100.0, 2, 0, 0, 0, 1, 0],
 9688412: [0, 0, 0, 100.0, 3, 1, 1, 0, 1, 0],
 9751070: [1, 0, 0, 100.0, 2, 1, 1, 0, 1, -1],
 9786655: [0, 0, 0, 100.0, 3, 0, 0, 0, 1, 0],
 9795814: [0, 1, 0, nan, 3, 

In [58]:
# Calculate the score by summing up all values
for keys, values in temp.items():
    risk_score = values[4] + values[1] + values[0] + values[5] + values[6] + values[7] + values[8] + values[9]
    temp[keys].append(risk_score)

# The final result

In [59]:
risk_df = pd.DataFrame(columns=['User ID', 'Completed sim form', 'Passed training', 'Used phishing button', 'Test score', 'Time since training', 'Viewed images', 'Clicked link', 'Opened attachment', 'Reported sim', 'Watched feedback', 'Individual Risk Score'])

In [60]:
# Append data into the new dataframe
for k,v in temp.items():
    tmp = {
        'User ID': k, 
        'Completed sim form': v[0], 
        'Passed training': v[1], 
        'Used phishing button': v[2], 
        'Test score': v[3], 
        'Time since training': v[4], 
        'Viewed images': v[5], 
        'Clicked link': v[6], 
        'Opened attachment': v[7], 
        'Reported sim': v[8], 
        'Watched feedback': v[9], 
        'Individual Risk Score': v[10]
    }
    
    risk_df = risk_df.append(tmp, ignore_index=True)

In [61]:
# Format User ID as integer
risk_df['User ID'] = risk_df['User ID'].astype(int)

In [62]:
# Placeholder status
risk_df['Status'] = 'green'

In [64]:
# Set the correct status
risk_df.loc[(risk_df['Individual Risk Score'] > green) & (risk_df['Individual Risk Score'] <= yellow), 'Status'] = 'yellow'
risk_df.loc[(risk_df['Individual Risk Score'] > yellow) & (risk_df['Individual Risk Score'] <= red), 'Status'] = 'red'

In [65]:
risk_df['Status'].value_counts()

yellow    17581
green     14730
Name: Status, dtype: int64

In [66]:
risk_df.head()

,User ID,Completed sim form,Passed training,Used phishing button,Test score,Time since training,Viewed images,Clicked link,Opened attachment,Reported sim,Watched feedback,Individual Risk Score,Status
0,6677201,0.0,0.0,0.0,100.0,3.0,0.0,0.0,0.0,1.0,0.0,4.0,green
1,6812989,0.0,0.0,0.0,100.0,3.0,0.0,0.0,0.0,1.0,0.0,4.0,green
2,6812994,0.0,0.0,0.0,100.0,2.0,0.0,1.0,0.0,1.0,0.0,4.0,green
3,7469741,0.0,0.0,0.0,100.0,3.0,0.0,0.0,0.0,1.0,0.0,4.0,green
4,8577487,1.0,0.0,0.0,100.0,3.0,1.0,1.0,0.0,1.0,-1.0,6.0,yellow


# More data exploration

In [67]:
# Total users in the dataset
n_users = risk_df.shape[0]
n_users

32311

In [68]:
# How many passed the training, didn't fail the simulation and reported the simulation email
passed_not_click = risk_df.loc[(risk_df['Passed training'] == 0.0) & (risk_df['Reported sim'] == -1.0) & (risk_df['Clicked link'] == 0.0)]['User ID'].count()
passed_not_click

466

In [69]:
percentage_actually_learned = (passed_not_click/n_users)*100
percentage_actually_learned

1.4422332951626382

In [70]:
# Phishing report, break down by type
phishing_report['Type'].value_counts()

Spam                 720
Internal Phishing    162
RealPhishing          78
Attachments           19
Name: Type, dtype: int64

In [71]:
phishing_report['Sender'].unique().shape

(491,)

In [72]:
# How many unique users have reported anything
unique_used_button = (491/n_users)*100

In [73]:
# How many reported emails are actually real
reporting_accuracy = (78/979)*100

In [74]:
# Add a country field
risk_df.insert(1, 'Country', '')

In [75]:
for k,v in sim_countries.items():
    risk_df.loc[risk_df['User ID'] == k, 'Country'] = v

In [76]:
# Save the file
risk_df.to_csv('ind_scores.csv')

### Save some interesting metrics

In [77]:
percentages = pd.DataFrame(columns=['Internalized', 'Used phishing report button', 'Phishing report accuracy', 'Average Individual Risk Score', 'Individual Risk Score std'])

In [78]:
data_dict = {
    'Internalized': round(percentage_actually_learned, 4), 
    'Used phishing report button': round(unique_used_button, 4), 
    'Phishing report accuracy': round(reporting_accuracy, 4),
    'Average Individual Risk Score': round(risk_df['Individual Risk Score'].mean(), 4), 
    'Individual Risk Score std': round(risk_df['Individual Risk Score'].std(), 4)
}

In [79]:
percentages = percentages.append(data_dict, ignore_index=True)

In [80]:
# Save the file
percentages.to_csv('percentages.csv')